# Perceptrón Unicapa (TLU) — Clasificación binaria de género en Twitter

**Taller 1 — Deep Learning (2026-10)**

**Modelo:** Perceptrón Unicapa con función escalón (Threshold Logic Unit)

**Objetivo:** Clasificar usuarios de Twitter como `male (0)` o `female (1)`.

Los datos ya fueron preprocesados en `Taller1DL.ipynb` (secciones 1-5) y se cargan desde archivos `.npy`.

## 1. Carga de datos preprocesados

Los archivos `.npy` contienen las matrices ya:
- Imputadas (nulos tratados)
- Escaladas (StandardScaler para numéricas)
- Codificadas (OneHotEncoder para categóricas)
- Particionadas (60% train, 20% val, 20% test)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar matrices preprocesadas generadas en Taller1DL.ipynb
X_train = np.load("X_train.npy")
X_val   = np.load("X_val.npy")
X_test  = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_val   = np.load("y_val.npy")
y_test  = np.load("y_test.npy")

# Verificar dimensiones
print("="*50)
print("  DIMENSIONES DE LOS DATOS")
print("="*50)
print(f"  Train: X={X_train.shape}, y={y_train.shape}")
print(f"  Val:   X={X_val.shape},   y={y_val.shape}")
print(f"  Test:  X={X_test.shape},  y={y_test.shape}")
print(f"\n  Número de features: {X_train.shape[1]}")
print(f"  Clases únicas: {np.unique(y_train)}  (0=male, 1=female)")

# Verificar distribución en cada conjunto
print(f"\n  Train → male: {np.sum(y_train==0)}, female: {np.sum(y_train==1)}")
print(f"  Val   → male: {np.sum(y_val==0)},   female: {np.sum(y_val==1)}")
print(f"  Test  → male: {np.sum(y_test==0)},  female: {np.sum(y_test==1)}")

  DIMENSIONES DE LOS DATOS
  Train: X=(7736, 9), y=(7736,)
  Val:   X=(2579, 9),   y=(2579,)
  Test:  X=(2579, 9),  y=(2579,)

  Número de features: 9
  Clases únicas: [0 1]  (0=male, 1=female)

  Train → male: 3716, female: 4020
  Val   → male: 1239,   female: 1340
  Test  → male: 1239,  female: 1340


## 2. ¿Qué es el Perceptrón Unicapa?

El **perceptrón** es el modelo de red neuronal más simple que existe. Fue propuesto por Frank Rosenblatt en 1958.

### Arquitectura:
```
x₁ ──→ w₁ ──┐
x₂ ──→ w₂ ──┤
x₃ ──→ w₃ ──┼──→ Σ(wᵢxᵢ) + b ──→ f(z) ──→ ŷ ∈ {0, 1}
 ⋮           │
xₙ ──→ wₙ ──┘
```

### Componentes:
- **Entradas (x):** Las features preprocesadas (N variables)
- **Pesos (w):** Un peso por cada feature, aprendido durante el entrenamiento
- **Bias (b):** Un sesgo que desplaza la frontera de decisión
- **Función de activación:** Escalón (step / TLU)

### Función escalón (Threshold Logic Unit):

$$\hat{y} = \begin{cases} 1 & \text{si } \sum_{i=1}^{n} w_i x_i + b \geq 0 \\ 0 & \text{si } \sum_{i=1}^{n} w_i x_i + b < 0 \end{cases}$$

### Regla de actualización del perceptrón:
Para cada muestra mal clasificada:
$$w_i \leftarrow w_i + \eta \cdot (y - \hat{y}) \cdot x_i$$

Donde $\eta$ es la **tasa de aprendizaje**.

### Limitación clave:
Solo puede resolver problemas **linealmente separables**. No puede aprender fronteras de decisión curvas o complejas.

## 3. Entrenamiento del Perceptrón

Usamos `sklearn.linear_model.Perceptron` que implementa exactamente el algoritmo del perceptrón clásico:
- **Sin capas ocultas** (unicapa)
- **Función escalón** como activación
- **Regla del perceptrón** para actualizar pesos

### Hiperparámetros:
| Parámetro | Valor | Descripción |
|---|---|---|
| `max_iter` | 1000 | Máximo de pasadas completas por los datos (épocas) |
| `eta0` | 0.01 | Tasa de aprendizaje (qué tanto ajusta los pesos) |
| `tol` | 1e-3 | Si la pérdida no mejora más que esto, para |
| `early_stopping` | True | Detiene si no mejora en validación interna |
| `n_iter_no_change` | 10 | Épocas consecutivas sin mejora para detenerse |
 

In [4]:
from sklearn.linear_model import Perceptron

# ============================================
# 3) Crear y entrenar el Perceptrón
# ============================================

perceptron = Perceptron(
    max_iter=1000,           # Máximo de épocas
    eta0=0.01,               # Tasa de aprendizaje
    tol=1e-3,                # Tolerancia para convergencia
    random_state=42,         # Reproducibilidad
    early_stopping=True,     # Parar si no mejora
    validation_fraction=0.2, # 20% del train para validación interna
    n_iter_no_change=10      # Épocas sin mejora para parar
)

# Entrenar SOLO con datos de entrenamiento
perceptron.fit(X_train, y_train)

# Información del modelo entrenado
print("="*50)
print("  MODELO ENTRENADO")
print("="*50)
print(f"  Épocas ejecutadas:    {perceptron.n_iter_}")
print(f"  Número de features:   {X_train.shape[1]}")
print(f"  Shape de pesos (w):   {perceptron.coef_.shape}")
print(f"  Bias (b):             {perceptron.intercept_[0]:.6f}")
print(f"  Pesos ≠ 0:            {np.sum(perceptron.coef_ != 0)} de {perceptron.coef_.shape[1]}")

  MODELO ENTRENADO
  Épocas ejecutadas:    13
  Número de features:   9
  Shape de pesos (w):   (1, 9)
  Bias (b):             0.010000
  Pesos ≠ 0:            8 de 9


## 4. Predicciones

El perceptrón aplica la función escalón: para cada muestra calcula $\mathbf{w} \cdot \mathbf{x} + b$ y si es ≥ 0 predice 1 (female), si es < 0 predice 0 (male).

In [5]:
# ============================================
# 4) Predicciones en los 3 conjuntos
# ============================================

y_train_pred = perceptron.predict(X_train)
y_val_pred   = perceptron.predict(X_val)
y_test_pred  = perceptron.predict(X_test)

# Verificación rápida
print("="*50)
print("  MUESTRA DE PREDICCIONES (Test, primeras 20)")
print("="*50)
print(f"  Real:      {y_test[:20]}")
print(f"  Predicho:  {y_test_pred[:20]}")
coincidencias = np.sum(y_test[:20] == y_test_pred[:20])
print(f"  Aciertos:  {coincidencias}/20")

  MUESTRA DE PREDICCIONES (Test, primeras 20)
  Real:      [0 0 1 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 0]
  Predicho:  [0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1]
  Aciertos:  14/20


## 5. Evaluación del modelo

### Métricas utilizadas:
- **Accuracy:** Proporción total de aciertos → $\frac{TP + TN}{Total}$
- **Precision:** De los que predijo como female, ¿cuántos realmente lo son? → $\frac{TP}{TP + FP}$
- **Recall:** De todas las female reales, ¿cuántas detectó? → $\frac{TP}{TP + FN}$
- **F1 Score:** Media armónica de Precision y Recall → $2 \cdot \frac{P \cdot R}{P + R}$

In [6]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report
)

def evaluar(y_true, y_pred, nombre):
    """Calcula e imprime las métricas de clasificación."""
    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec  = recall_score(y_true, y_pred)
    f1   = f1_score(y_true, y_pred)

    print(f"\n{'='*55}")
    print(f"  {nombre}")
    print(f"{'='*55}")
    print(f"  Accuracy:  {acc:.4f}  ({acc*100:.2f}%)")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print()
    print(classification_report(
        y_true, y_pred,
        target_names=["male (0)", "female (1)"]
    ))

    return {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}

In [7]:
# ============================================
# 5.1) Evaluar en cada conjunto
# ============================================

m_train = evaluar(y_train, y_train_pred, "TRAIN")
m_val   = evaluar(y_val,   y_val_pred,   "VALIDACIÓN")
m_test  = evaluar(y_test,  y_test_pred,  "TEST")


  TRAIN
  Accuracy:  0.5359  (53.59%)
  Precision: 0.5353
  Recall:    0.8119
  F1 Score:  0.6452

              precision    recall  f1-score   support

    male (0)       0.54      0.24      0.33      3716
  female (1)       0.54      0.81      0.65      4020

    accuracy                           0.54      7736
   macro avg       0.54      0.52      0.49      7736
weighted avg       0.54      0.54      0.49      7736


  VALIDACIÓN
  Accuracy:  0.5518  (55.18%)
  Precision: 0.5454
  Recall:    0.8254
  F1 Score:  0.6568

              precision    recall  f1-score   support

    male (0)       0.58      0.26      0.35      1239
  female (1)       0.55      0.83      0.66      1340

    accuracy                           0.55      2579
   macro avg       0.56      0.54      0.51      2579
weighted avg       0.56      0.55      0.51      2579


  TEST
  Accuracy:  0.5308  (53.08%)
  Precision: 0.5322
  Recall:    0.8007
  F1 Score:  0.6395

              precision    recall  f1-scor

### 5.2 Tabla comparativa entre conjuntos

In [10]:
# ============================================
# 5.2) Tabla resumen
# ============================================

resumen = pd.DataFrame({
    "Train": m_train,
    "Validación": m_val,
    "Test": m_test
}).T

print("\nPerceptrón Unicapa — Resumen de métricas")
print("="*55)
display(resumen.round(4))


Perceptrón Unicapa — Resumen de métricas


,Accuracy,Precision,Recall,F1
Train,0.5359,0.5353,0.8119,0.6452
Validación,0.5518,0.5454,0.8254,0.6568
Test,0.5308,0.5322,0.8007,0.6395
